# Yuan2.0

本 Notebook 展示了如何在 LangChain 中使用 [YUAN2 API](https://github.com/IEIT-Yuan/Yuan-2.0/blob/main/docs/inference_server.md)，通过 langchain.chat_models.ChatYuan2。

[*Yuan2.0*](https://github.com/IEIT-Yuan/Yuan-2.0/blob/main/README-EN.md) 是由 IEIT System 开发的新一代基础大语言模型。我们已发布了 Yuan 2.0-102B、Yuan 2.0-51B 和 Yuan 2.0-2B 这三个模型，并为其他开发者提供了预训练、微调和推理服务的相关脚本。Yuan2.0 基于 Yuan1.0，利用了更广泛的高质量预训练数据和指令微调数据集，以增强模型在语义、数学、推理、代码、知识等方面的理解能力。

## 入门

### 安装

首先，Yuan2.0 提供了一个与 OpenAI 兼容的 API，我们通过 OpenAI 客户端将 ChatYuan2 集成到 langchain 的 chat 模型中。

因此，请确保你的 Python 环境中已安装 openai 包。运行以下命令：

In [ ]:
%pip install --upgrade --quiet openai

### 导入所需模块
安装完成后，请将必要的模块导入到您的 Python 脚本中：

In [ ]:
from langchain_community.chat_models import ChatYuan2
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

### 设置您的 API 服务器
按照 [yuan2 openai api server](https://github.com/IEIT-Yuan/Yuan-2.0/blob/main/docs/Yuan2_fastchat.md) 的说明设置您的 OpenAI 兼容 API 服务器。
如果您在本地部署了 API 服务器，您可以将 `yuan2_api_key` 设置为 `"EMPTY"` 或任何您想要的值。
请确保 `yuan2_api_base` 设置正确。

In [ ]:
yuan2_api_key = "your_api_key"
yuan2_api_base = "http://127.0.0.1:8001/v1"

### 初始化 ChatYuan2 模型
以下是如何初始化聊天模型：

In [ ]:
chat = ChatYuan2(
    yuan2_api_base="http://127.0.0.1:8001/v1",
    temperature=1.0,
    model_name="yuan2",
    max_retries=3,
    streaming=False,
)

### 基本用法
像这样使用系统和用户消息来调用模型：

In [ ]:
messages = [
    SystemMessage(content="你是一个人工智能助手。"),
    HumanMessage(content="你好，你是谁？"),
]

In [ ]:
print(chat.invoke(messages))

### 基础用法（流式输出）
如需实现持续互动，请使用流式输出功能：

In [ ]:
from langchain_core.callbacks import StreamingStdOutCallbackHandler

chat = ChatYuan2(
    yuan2_api_base="http://127.0.0.1:8001/v1",
    temperature=1.0,
    model_name="yuan2",
    max_retries=3,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
messages = [
    SystemMessage(content="你是个旅游小助手。"),
    HumanMessage(content="给我介绍一下北京有哪些好玩的。"),
]

In [ ]:
chat.invoke(messages)

## 高级功能
### 配合异步调用使用

像这样，使用非阻塞调用来调用模型：

In [ ]:
async def basic_agenerate():
    chat = ChatYuan2(
        yuan2_api_base="http://127.0.0.1:8001/v1",
        temperature=1.0,
        model_name="yuan2",
        max_retries=3,
    )
    messages = [
        [
            SystemMessage(content="你是个旅游小助手。"),
            HumanMessage(content="给我介绍一下北京有哪些好玩的。"),
        ]
    ]

    result = await chat.agenerate(messages)
    print(result)

In [ ]:
import asyncio

asyncio.run(basic_agenerate())

### 与提示词模板结合使用

使用非阻塞调用并像这样使用聊天模板来调用模型：

In [ ]:
async def ainvoke_with_prompt_template():
    from langchain_core.prompts.chat import (
        ChatPromptTemplate,
    )

    chat = ChatYuan2(
        yuan2_api_base="http://127.0.0.1:8001/v1",
        temperature=1.0,
        model_name="yuan2",
        max_retries=3,
    )
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一个诗人，擅长写诗。"),
            ("human", "给我写首诗，主题是{theme}。"),
        ]
    )
    chain = prompt | chat
    result = await chain.ainvoke({"theme": "明月"})
    print(f"type(result): {type(result)}; {result}")

In [ ]:
asyncio.run(ainvoke_with_prompt_template())

### 在流式处理中使用异步调用
对于具有流式输出的非阻塞调用，请使用 `astream` 方法：

In [ ]:
async def basic_astream():
    chat = ChatYuan2(
        yuan2_api_base="http://127.0.0.1:8001/v1",
        temperature=1.0,
        model_name="yuan2",
        max_retries=3,
    )
    messages = [
        SystemMessage(content="你是个旅游小助手。"),
        HumanMessage(content="给我介绍一下北京有哪些好玩的。"),
    ]
    result = chat.astream(messages)
    async for chunk in result:
        print(chunk.content, end="", flush=True)

In [ ]:
import asyncio

asyncio.run(basic_astream())